Work in progress...

In [ ]:
import duckdb
import os
import pandas as pd

In [ ]:
# Directory where Parquet files are stored
parquet_dir = "./parquet_data"

In [ ]:
# Initialize DuckDB connection
con = duckdb.connect(database=':memory:')  # Use in-memory database
# con = duckdb.connect("azure_logs.db")

In [ ]:
parquet_file = os.path.join(parquet_dir,"insights-logs-auditlogs","20240619","data.parquet")

In [ ]:
con.execute(f"SELECT * FROM read_parquet('{parquet_file}') LIMIT 1").fetchall()

In [ ]:
df = con.execute(f"SELECT * FROM read_parquet('{parquet_file}') LIMIT 1").df()

# Print the column names
print(df.columns.tolist())

In [ ]:
df

con.close()

In [ ]:
# Directory where Parquet files are stored
auditlogs_dir = "./parquet_data/insights-logs-auditlogs"
signinlogs_dir = "./parquet_data/insights-logs-signinlogs"

# Initialize DuckDB connection
# con = duckdb.connect(database=':memory:')  # Use in-memory database for this example

con = duckdb.connect("azure_logs_2.db")



In [ ]:
auditlogs_combined_file = os.path.join(auditlogs_dir, "combined_table.parquet")
signinlogs_combined_file = os.path.join(signinlogs_dir, "combined_table.parquet")

con.execute(f"DROP TABLE IF EXISTS combined_table_auditlogs")
con.execute(f"DROP TABLE IF EXISTS combined_table_signinlogs")
con.execute(f"CREATE TABLE combined_table_auditlogs AS SELECT * FROM read_parquet('{auditlogs_combined_file}')")
con.execute(f"CREATE TABLE combined_table_signinlogs AS SELECT * FROM read_parquet('{signinlogs_combined_file}')")
    

In [ ]:
df = con.execute("DESCRIBE SELECT * FROM combined_table_auditlogs").df()
df

In [ ]:
df = con.execute("SELECT * FROM combined_table_auditlogs LIMIT 30").df()
df

In [ ]:
df = con.execute("DESCRIBE SELECT * FROM combined_table_signinlogs").df()
df

In [ ]:
df = con.execute("SELECT * FROM combined_table_signinlogs LIMIT 30").df()
df

In [ ]:
query = '''SELECT "time", "properties.initiatedBy.user.id", "properties.initiatedBy.user.displayName", "properties.initiatedBy.user.userPrincipalName", "properties.initiatedBy.user.ipAddress" FROM combined_table_auditlogs WHERE "properties.initiatedBy.user.id"='REDACTED' LIMIT 30;'''


# query = 'SELECT "time", "properties.initiatedBy.user.id", "properties.initiatedBy.user.displayName", "properties.initiatedBy.user.userPrincipalName", "properties.initiatedBy.user.ipAddress" FROM combined_table_auditlogs LIMIT 30'

df = con.execute(query).df()
df

In [ ]:
query = 'SELECT "time", "properties.userId", "properties.userPrincipalName", "properties.userDisplayName", "properties.ipAddress" FROM combined_table_signinlogs LIMIT 30'

df = con.execute(query).df()
df

In [ ]:
con.execute("DROP TABLE IF EXISTS sessions")

query = '''
CREATE TABLE sessions AS
SELECT
    ROW_NUMBER() OVER () AS session_id,
    MIN(CAST("time" AS TIMESTAMP)) AS start_time,
    "properties.userId" AS user_id,
    "properties.userPrincipalName" AS user_principal_name,
    "properties.ipAddress" AS ip_address,
    MIN(CAST("time" AS TIMESTAMP)) AS session_start,
    MAX(CAST("time" AS TIMESTAMP)) + INTERVAL '2 hour' AS session_end
FROM
    combined_table_signinlogs
GROUP BY
    user_id,
    user_principal_name,
    ip_address,
    DATE_TRUNC('hour', CAST("time" AS TIMESTAMP))
HAVING
    COUNT(*) > 1; 
'''

con.execute(query)

df = con.execute("SELECT * FROM sessions").df()
df

In [ ]:
query = '''
ALTER TABLE combined_table_signinlogs ADD COLUMN sessionId VARCHAR;

UPDATE combined_table_signinlogs AS signin
SET sessionId = (
    SELECT sess.session_id
    FROM sessions AS sess
    WHERE signin."properties.userId" = sess.user_id
      AND signin."properties.userPrincipalName" = sess.user_principal_name
      AND signin."properties.ipAddress" = sess.ip_address
      AND CAST(signin."time" AS TIMESTAMP) BETWEEN sess.session_start AND sess.session_end
)
'''
con.execute(query)

In [ ]:
query='''
ALTER TABLE combined_table_auditlogs ADD COLUMN sessionId VARCHAR;

UPDATE combined_table_auditlogs AS audit
SET sessionId = (
    SELECT sess.session_id
    FROM sessions AS sess
    WHERE audit."properties.initiatedBy.user.id" = sess.user_id
      AND audit."properties.initiatedBy.user.userPrincipalName" = sess.user_principal_name
      AND audit."properties.initiatedBy.user.ipAddress" = sess.ip_address
      AND CAST(audit."time" AS TIMESTAMP) BETWEEN sess.session_start AND sess.session_end
)
'''
con.execute(query)

In [ ]:
# query = '''SELECT * FROM combined_table_signinlogs WHERE "properties.userId"='REDACTED' and "properties.uniqueTokenIdentifier"='REDACTED' LIMIT 30;'''

# query = '''SELECT "sessionId", "time", "properties.initiatedBy.user.id", "properties.initiatedBy.user.displayName", "properties.initiatedBy.user.userPrincipalName", "properties.initiatedBy.user.ipAddress" FROM combined_table_logs WHERE "properties.initiatedBy.user.id"='REDACTED' LIMIT 30;'''

# query = 'SELECT "sessionId", "time", "properties.initiatedBy.user.id", "properties.initiatedBy.user.displayName", "properties.initiatedBy.user.userPrincipalName", "properties.initiatedBy.user.ipAddress" FROM combined_table_auditlogs LIMIT 30'

# query = '''SELECT "properties.correlationId" FROM combined_table_auditlogs'''

# query = '''SELECT * FROM combined_table_signinlogs WHERE "properties.correlationId"='REDACTED' '''

query = '''SELECT signins."operationName", audits."properties.operationName" FROM combined_table_signinlogs AS signins JOIN combined_table_auditlogs as audits ON signins.correlationId = audits."properties.correlationId" '''

df = con.execute(query).df()
df.value_counts()

In [ ]:
query = 'SELECT "sessionId", "time", "properties.userId", "properties.userPrincipalName", "properties.userDisplayName", "properties.ipAddress" FROM combined_table_signinlogs LIMIT 30'

df = con.execute(query).df()
df

In [ ]:
con.close()